`Time.now`是一个间接输入。首先我们引用了`Time`类，然后把`#now`消息发送给`Time`。我们想要的其实是方法`now`的返回值，而不是`Time`常量本身。任何时候为了得到方法返回值，而向其他对象发送消息，其实就是在使用间接输入。

间接输入层次越深，代码的耦合程度就越高

In [1]:
Place = Struct.new(:index, :name, :prize)

Place

In [3]:
first = Place.new(0, "first", "Peasant's Quest game")

#<struct Place index=0, name="first", prize="Peasant's Quest game">

In [4]:
second = Place.new(1, "second", "Limozeen Album")

#<struct Place index=1, name="second", prize="Limozeen Album">

In [9]:
thrid = Place.new(2, "third", "butter-da")

#<struct Place index=2, name="third", prize="butter-da">

In [10]:
winners = ["a", "b"]

["a", "b"]

In [11]:
[first, second, thrid].each do |place|
  p "In #{place.name} place, #{winners[place.index]}"
  p "you wind: #{place.prize}"
end

"In first place, a"
"you wind: Peasant's Quest game"
"In second place, b"
"you wind: Limozeen Album"
"In third place, "
"you wind: butter-da"


[#<struct Place index=0, name="first", prize="Peasant's Quest game">, #<struct Place index=1, name="second", prize="Limozeen Album">, #<struct Place index=2, name="third", prize="butter-da">]

In [15]:
class EmacsConfigFile
  def initialize
    @filename = "#{ENV['HOME']}/.vimrc"
  end
  
  def to_path
    @filename
  end
end

:to_path

In [16]:
emacs_config = EmacsConfigFile.new

#<EmacsConfigFile:0x00000055c47320e8 @filename="/home/demouser/.vimrc">

In [17]:
File.open(emacs_config).lines.count

<main>: warning: IO#lines is deprecated; use #each_line instead


740

这是因为，`EmacsConfig`类定义了转换方法`#to_path`，而`File#open`又会在参数对象上调用`#to_path`方法，以便得到文件名字符串。这样一来，这个非字符串对象也能工作得很好。

下面的例子可以说明两者（隐式转化和显示转化的区别）。`Time`对象并非`String`，有太多方式可以将`Time`转化成`String`了。另外，这两种类型还是不想管的，因此`Time`只预定义了显式的类型转换方法`#to_s`，而没有定义隐式版的`#to_str`

In [20]:
now = Time.now
now.respond_to?(:to_s)
now.to_s
now.respond_to?(:to_str)

false

In [19]:
"hello, " + "world"

"hello, world"